# Mnist Classification using NN

* Mnist Classification - The Hello World of deep learning ^

##### Imports

In [12]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

##### Load Mnist Dataset

In [13]:
# with_info also gives us the meta data
# as_supervised returns us the dataset in image, label pairs
mnist_data, mnist_info = tfds.load(name='mnist', with_info=True, as_supervised=True)

In [14]:
mnist_train, mnist_test = mnist_data['train'], mnist_data['test']

In [15]:
mnist_info

tfds.core.DatasetInfo(
    name='mnist',
    full_name='mnist/3.0.1',
    description="""
    The MNIST database of handwritten digits.
    """,
    homepage='http://yann.lecun.com/exdb/mnist/',
    data_path='C:\\Users\\Palmg\\tensorflow_datasets\\mnist\\3.0.1',
    file_format=tfrecord,
    download_size=11.06 MiB,
    dataset_size=21.00 MiB,
    features=FeaturesDict({
        'image': Image(shape=(28, 28, 1), dtype=uint8),
        'label': ClassLabel(shape=(), dtype=int64, num_classes=10),
    }),
    supervised_keys=('image', 'label'),
    disable_shuffling=False,
    splits={
        'test': <SplitInfo num_examples=10000, num_shards=1>,
        'train': <SplitInfo num_examples=60000, num_shards=1>,
    },
    citation="""@article{lecun2010mnist,
      title={MNIST handwritten digit database},
      author={LeCun, Yann and Cortes, Corinna and Burges, CJ},
      journal={ATT Labs [Online]. Available: http://yann.lecun.com/exdb/mnist},
      volume={2},
      year={2010}
    }""",
)

In [17]:
# 10% of train data becomes validation data
validation_size = 0.1 * mnist_info.splits['train'].num_examples
validation_size = tf.cast(validation_size, tf.int64)
validation_size

<tf.Tensor: shape=(), dtype=int64, numpy=6000>

In [18]:
test_size = mnist_info.splits['test'].num_examples
test_size = tf.cast(test_size, tf.int64)
test_size

<tf.Tensor: shape=(), dtype=int64, numpy=10000>

##### Scaling

In [19]:
# Function to scale all pixel values from 0-255 to 0-1
def scale(image, label):
    image = tf.cast(image, tf.float32)
    image = image / 255.
    return image, label